In [22]:
import sys
sys.path.append("..") # this adds to path parent directory in order to import utils file
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from tqdm import tqdm
import numpy as np
from sklearn import metrics
from IPython.display import clear_output, Image, display
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.decomposition import PCA
import hyperopt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from sklearn.neighbors import kneighbors_graph
import igraph as ig
import louvain
import time
import pickle
import umap
## Import custom utils
import gmm_utils
import hyperopt_utils

In [23]:
%load_ext autoreload
import importlib
importlib.reload(gmm_utils)
importlib.reload(hyperopt_utils)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
elbowAnalysis
silhouetteAnalyis


<module 'hyperopt_utils' from '../hyperopt_utils.py'>

## Hyperopt utils

In [24]:
dataset = 'brainCIDR'
df, truth = gmm_utils.loadData(dataset)

print(df.shape, truth.shape, truth.clusters.unique())
params={}
params['dataset'] = 'brainCIDR'
params['minCellsPerGene'] = 0
params['minGeneDispersion'] = 0
params['log'] = True # True, False
params['scaler'] = 'standardScaleCells'# 
params['pca_comp'] = 10 #range (3, 300)
params['doUmap'] = True #range (3, 300)
params['umap_comp'] = 3
params['nb_neighbors'] =10 #3 -15
gmm_utils.runLouvain(params);

(420, 22085) (420, 2) [3 4 1 6 2 7 5 0]


({'dataset': 'brainCIDR',
  'doUmap': True,
  'log': True,
  'minCellsPerGene': 0,
  'minGeneDispersion': 0,
  'nb_neighbors': 10,
  'pca_comp': 10,
  'randIndex': 0.3896711401407734,
  'scaler': 'standardScaleCells',
  'umap_comp': 3},
 array([ 3,  1, 10, 10,  3,  1,  3, 10, 10,  3, 10,  3,  8,  3,  1,  3,  1,
         1,  4,  1,  4,  4,  3,  3,  4,  3,  3,  3,  3,  4, 10,  3,  3,  3,
         1, 10,  3,  3, 10, 10,  8, 10,  4,  1,  3,  3,  3,  4,  1,  1,  4,
         3,  3,  4,  1,  4,  4,  3,  1,  3,  4,  1, 10,  1, 10, 10,  1,  4,
         1,  4,  3,  4, 10,  1,  4,  4, 10,  1,  4, 10,  3,  4,  4,  1,  3,
         4,  1,  4,  1, 10,  2,  2,  0, 13, 13, 11, 16, 11, 11,  0,  0, 16,
         6, 13, 16,  0,  7,  0,  6, 13,  7, 16,  0, 15,  6,  0,  0, 13, 17,
        17, 13, 12, 12, 13, 17,  7, 17,  5,  0, 12,  5,  7, 17,  7, 13, 17,
         8,  8,  1,  8,  8,  8,  8,  8,  8,  1,  8,  8,  8,  8,  1,  8,  8,
         8,  1,  8,  1,  8,  1,  1,  1,  8,  4,  1,  4,  1,  4,  1, 10,  1,
   

In [25]:

def runHyperopt(filename, space, max_evals = 2, restart = False):
    # Define function to optimise
    def evaluateLouvain(args):
        try:
            resultDict, _ = gmm_utils.runLouvain(args)
        except:
            return { 'status' : hyperopt.STATUS_FAIL}

        print(f'>> Result: {resultDict["randIndex"]}')
        ret = {
            'loss' : -resultDict['randIndex']
            ,'status' : STATUS_OK
            ,'eval_time' : time.time()        
        }
        return ret

    trials = hyperopt_utils.getTrials(filename ,restart = restart )
    evals_per_epoch = 10
    for e in range(len(trials), max_evals, evals_per_epoch):
        best = fmin(evaluateLouvain
                    ,space
                    ,algo=tpe.suggest 
                    ,max_evals= e + evals_per_epoch
                    ,trials=trials)
        print('Index ', e)
        pickle.dump(trials, open(filename, 'wb'))
    return trials

## Brain CIDR

In [29]:
dataset = 'brainCIDR'
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}

filename = f'{dataset}_louvain_trials.pkl'

In [30]:
trials=hyperopt_utils.getTrials(filename ,restart = False )

Creating new trials...


In [ ]:
trials = runHyperopt(filename, space, max_evals = 500, restart = True);

Creating new trials...
>> Result: 0.3823085687972271
>> Result: 0.26569598332185074
>> Result: 0.42681363257444055
>> Result: 0.41937512893833306
>> Result: 0.5682008392900804
>> Result: 0.35089113970218055


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


>> Result: 0.6435556351100351
>> Result: 0.3233628764992152
>> Result: 0.45479620648895835
>> Result: 0.527020448578572
Index  0
>> Result: 0.44317862756972254
>> Result: 0.39861367397288233
>> Result: 0.3892968675720729
>> Result: 0.37831073005945337
>> Result: 0.4961602750698821


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


>> Result: 0.7596723111604312
>> Result: 0.5483240288797667
>> Result: 0.34335539098249956


...

>> Result: 0.47351558085713386


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


>> Result: 0.5490597992742366
>> Result: 0.5853219546839146


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


>> Result: 0.7308323089666864
>> Result: 0.3891154409580249
>> Result: 0.7643200271042186
>> Result: 0.7493201642890226
>> Result: 0.6602662794678176
>> Result: 0.570075995461746
>> Result: 0.5440569319691174
Index  60
>> Result: 0.4407932202106022
>> Result: 0.7836092125332547
>> Result: 0.6715425064973602
>> Result: 0.676404312700071
>> Result: 0.7890512703035873
>> Result: 0.5518334532818321
>> Result: 0.6695364164156907
>> Result: 0.6879806555422511
>> Result: 0.770109012292583
>> Result: 0.7801644359143579
Index  70
>> Result: 0.8413495159018819
>> Result: 0.67367144760814
>> Result: 0.7876663083099568
>> Result: 0.46523660525485255
>> Result: 0.5174730646445744
>> Result: 0.7919127987354188
>> Result: 0.5680880528552832
>> Result: 0.5303652658573142
>> Result: 0.7512775297348602
>> Result: 0.7901441510891695
Index  80
>> Result: 0.7680808878055183
>> Result: 0.7588220005031866
>> Result: 0.4267262887300326
>> Result: 0.7368367805963439
>> Result: 0.6645374004605826
>> Result: 0.4

In [ ]:
summaryDf = hyperopt_utils.getResultsAsDf(trials, space)
summaryDf.sort_values(by='result', ascending =True).head()

In [ ]:
summaryDf.to_pickle(f'{dataset}_louvain_df.pkl')

In [ ]:
summaryDf = pd.read_pickle(f'{dataset}_louvain_df.pkl')
gmm_utils.plotBestPrediction(summaryDf, dataset)

## PancreaticIsletCIDR

In [ ]:
dataset= 'pancreaticIsletCIDR'
df, truth = gmm_utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())

In [ ]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}

filename = f'{dataset}_louvain_trials.pkl'

In [ ]:
trials=hyperopt_utils.getTrials(filename ,restart = False )

In [ ]:
trials = runHyperopt(filename, space, max_evals = 500, restart = True);

In [ ]:
summaryDf = hyperopt_utils.getResultsAsDf(trials, space)
summaryDf.sort_values(by='result', ascending =True).head()

In [ ]:
summaryDf.to_pickle(f'{dataset}_louvain_df.pkl')

In [ ]:
summaryDf = pd.read_pickle(f'{dataset}_louvain_df.pkl')
gmm_utils.plotBestPrediction(summaryDf, dataset, pca_comp = 4)

## Deng

In [ ]:
dataset= 'deng'
df, truth = gmm_utils.loadData(dataset)
umap2D = gmm_utils.getUmap(df, pca_comp = 10)
print(df.shape, truth.shape, truth.clusters.unique())

In [ ]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}

filename = f'{dataset}_louvain_trials.pkl'

In [ ]:
trials=hyperopt_utils.getTrials(filename ,restart = False )

In [ ]:
trials = runHyperopt(filename, space, max_evals = 500, restart = True);

In [ ]:
summaryDf = hyperopt_utils.getResultsAsDf(trials, space)
summaryDf.sort_values(by='result', ascending =True).head()

In [ ]:
summaryDf.to_pickle(f'{dataset}_louvain_df.pkl')

In [ ]:
summaryDf = pd.read_pickle(f'{dataset}_louvain_df.pkl')
gmm_utils.plotBestPrediction(summaryDf, dataset, pca_comp = 4)

In [ ]:
bestBic, bestAic, bestSil = optimalNbClustersGMM(pc, params['min_clusters'], params['max_clusters'], 2)

In [ ]:
bestBic, bestAic, bestSil

In [ ]:
for n_clust in n_clusters:
    model = GaussianMixture(n_clust, covariance_type ='full', random_state = 0).fit(pc)
    clusters = model.predict(pc)
    score = adjusted_rand_score(truth.clusters.tolist(), clusters)
    print(f"For {n_clust} clusters, score : {score}")

## Evaluation

In [ ]:
model = GaussianMixture(8, covariance_type ='full', random_state = 0).fit(pc)
clusters = model.predict(pc)
score = adjusted_rand_score(truth.clusters.tolist(), clusters)
score

In [ ]:
truth.clusters.value_counts().shape

In [ ]:
dataset = 'brainCIDR'
df, truth = gmm_utils.loadData(dataset)
umap2D = gmm_utils.getUmap(df, pca_comp = 10)
print(df.shape, truth.shape, truth.clusters.unique())
params={}
params['dataset'] = 'brainCIDR'
params['minCellsPerGene'] = 0
params['minGeneDispersion'] = 0
params['log'] = True # True, False
params['scaler'] = 'standardScaleCells'# 
params['pca_comp'] = 10 #range (3, 300)
params['nb_clusters'] =8 #3 -15
gmm_utils.run(params);